<a href="https://colab.research.google.com/github/Shreyaswini-M-R/Mini-Project/blob/main/Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Image Classification

In [ ]:
# Import Libraries
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
# 1. Gather Data

# By Using Bing Image Downloader
!pip install bing-image-downloader

In [ ]:
# Create a new folder
!mkdir images

In [ ]:
# Download Images
from bing_image_downloader import downloader
downloader.download("blueberry fruit",limit=30,output_dir='images',adult_filter_off=True)
downloader.download("puma animal",limit=30,output_dir='images',adult_filter_off=True)
downloader.download("lotus flower",limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
# 2. Preprocess Data

# Resizing and Flattening images
# Iterating through the images

from skimage.io import imread
from skimage.transform import resize

output = []
images = []
flat_data = []

DATADIR = '/content/images'
CATEGORIES = ['blueberry fruit','puma animal','lotus flower']

for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resized = resize(img_array,(150,150,3))     # Resizing the images
    flat_data.append(img_resized.flatten())         # Flattening the images
    images.append(img_resized)
    output.append(class_num)
flat_data = np.array(flat_data)
output = np.array(output)
images = np.array(images)

In [ ]:
# Split data into Training and Texting
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,output,test_size=0.5,random_state=50)

In [ ]:
# 3. Algorithm

# Using GridSearchCV
# Using SVM Algorithm

from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]

svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
# 4. Evaluate Model

from sklearn.metrics import accuracy_score,confusion_matrix
y_pred = clf.predict(x_test)
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

In [ ]:
# 5. Predict for a new Image

import pickle
pickle.dump(clf,open('img_model.p','wb'))
model = pickle.load(open('img_model.p','rb'))

flat_data = []
url = input('Enter your URL')
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = CATEGORIES[y_out[0]]
print(f' PREDICTED OUTPUT: {y_out}')